In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.text_cell_render.rendered_html{font-size:15pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:18pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:18pt;padding:5px;}
table.dataframe{font-size:18px;}
</style>
"""))

In [ ]:
#  요기요 웹크롤링

In [ ]:
# 네이버/구글 이미지 검색 수집 (이미지, 날짜, 장소 등 가능한지)

In [11]:
from dotenv import load_dotenv
import os
import sys
import urllib.request
import urllib.parse
import pandas as pd
import json
load_dotenv()

True

` 방법 
1.네이버 이미지 관련 크롤링 
2.이미지 url, 게시일자 추출
3.제목,본문에서 <가게명,위치,가격 등 패턴 정규식 추출>
4.이미지다운로드
5.구조화된 데이터로 정리 (csv)
`

In [28]:
# 이미지 검색 
# 환경변수 키 불러오기
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

# 검색어 인코딩
encText = urllib.parse.quote('서울 짜장면')
url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display=100" # JSON 결과 

# Request 객체 생성 + 헤더 추가 
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

# 응답 받기
with urllib.request.urlopen(request) as response:
    result = response.read().decode('utf-8')

# JSON 파싱
data = json.loads(result)
items = data['items']

# 리스트로 정리 
items_list = []
for item in items:
    items_list.append([
        item.get('title').replace('<b>','').replace('</b>',''),
        item.get('link'),
        item.get('description').replace('<b>','').replace('</b>',''),
        item.get('postdate')
    ])
    
# DataFrame 생성
seoul_black_noodle_100 = pd.DataFrame(items_list,columns=['title','link','postdate','description'])
soeul_black_noodle_100

# 데이터 프레임 저장
df.to_csv('data/test1_seoul_black_noodle_100.csv',encoding='utf-8-sig' ,index=False)

# 데이터 추출
# import json
# items = response.json()['items']

In [23]:
seoul_black_noodle_100.iloc[:,2:3]

,postdate
0,업체에 서비스를 제공받아 직접 방문하여 작성한 리뷰입니다 맛돌이들이라면 모두 아는 ...
1,잘 먹었습니다. 또 오겠습니다. 주관적 평점 5 #팔공 #중화요리팔공 #관악구맛집 ...
2,#광화문맛집 #서울광화문맛집 #광화문역맛집 #광화문주변맛집 #광화문역점심맛집 #경복...
3,"서촌 맛집 추천, 서울 경복궁 서촌마을 짜장면 중식 혼밥 맛있는 식당 서울 서촌 마..."
4,서울 짜장면 맛집! 서울 동대문 창신동 맛집! 전통 수타면 방식의 동대문 일품손짜장...
...,...
95,#짜장면 #간짜장 #짜장면맛집 #서울짜장면맛집 #중식당 #블랙데이 #짜장면데이 #홍...
96,"중화요리 생각날 때 맛있게 먹고 싶을 때 최근에 먹어본 짜장면, 짬뽕, 탕수육 서울..."
97,"채끝살 김치짜장면 서울시스터즈 김치짜장면, 채끝살, 트러플오일,양파 채끝살 김치짜장..."
98,구매는 여기서 https://www.rodemfoodshop.com/goods/go...


네이버 지도에서 가져오는 방법

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re

In [4]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")  # 창 안 띄우고 싶다면 주석 해제

# ▶ 크롬 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

# ▶ 네이버 지도 서울 음식점 검색 URL
driver.get("https://map.naver.com/p/search/서울 신림동 음식점")
time.sleep(5)

# ▶ 검색 결과 iframe 진입
driver.switch_to.frame("searchIframe")
time.sleep(2)

# place_ids = set()

# # ▶ 페이지 반복 크롤링
# for page in range(1, 20):  # 🔁 필요시 더 늘리기 (예: 100 페이지까지)
#     print(f"[INFO] 페이지 {page} 처리 중...")
#     time.sleep(2)

#     items = driver.find_elements(By.CSS_SELECTOR, 'a.place_bluelink')

#     for item in items:
#         href = item.get_attribute('href')
#         match = re.search(r'place/(\d+)', href)
#         if match:
#             place_ids.add(match.group(1))

#     # ▶ 다음 페이지 버튼 클릭
#     try:
#         next_btn = driver.find_element(By.CSS_SELECTOR, 'a.pagination.next')
#         if 'disabled' in next_btn.get_attribute('class'):
#             break
#         next_btn.click()
#     except:
#         print("[INFO] 다음 페이지 없음 또는 에러.")
#         break

# # ▶ 드라이버 종료
# driver.quit()

# # ▶ 결과 저장
# with open("seoul_place_ids.txt", "w") as f:
#     for pid in sorted(place_ids):
#         f.write(f"{pid}\n")

# print(f"[DONE] 총 수집된 placeId: {len(place_ids)}개")

# 네이버 지도에서 사진을 통해 업체별 사진 받아는 방법인데 아래꺼 좀 더 정리하면 될듯... 

In [8]:
driver = webdriver.Chrome()
place_url = "https://naver.me/5vcHMukK"  # 예: 스타벅스 종로점
driver.get(place_url)
time.sleep(5)

# 2. iframe 내부 접근
driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, "iframe#entryIframe"))
time.sleep(2)

# 3. "사진" 탭 클릭
try:
    photo_tab = driver.find_element(By.CSS_SELECTOR, 'a[href*="photo"]')
    photo_tab.click()
    time.sleep(3)
except Exception as e:
    print("사진 탭 클릭 실패:", e)

# 4. 스크롤해서 이미지 로딩 더 하기
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

# 5. 이미지 태그 수집
img_elements = driver.find_elements(By.CSS_SELECTOR, "img")  # 전체 이미지 태그 수집
os.makedirs("naver_images/starbucks_photos", exist_ok=True)

# 6. 이미지 저장
for i, img in enumerate(img_elements):
    try:
        src = img.get_attribute("src")
        if src and src.startswith("https") and "place" in src:
            urllib.request.urlretrieve(src, f"naver_images/starbucks_photos/img_{i}.jpg")
    except Exception as e:
        print(f"이미지 저장 실패 {i}: {e}")

driver.quit()
print("완료!")

완료!


# 신림동 음식점 정보 추출기 (이름,주소, placeld)


In [4]:
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
 
from time import sleep
import random
import re
 
from selenium import webdriver
import sys
 
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

# 1. 네이버 지도 검색 페이지 열기 (신림동 음식점)
URL = "https://map.naver.com/p/search/신림동 햄버거"
driver.get(url=URL)

# 2. 검색 결과 iframe 진입 
driver.switch_to.frame("searchIframe")

# 2. 페이지 숫자 체크
while(True):
    switch_left()
 
    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div[3]/div[2]/a[7]"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff73c2a6f95+76917]
	GetHandleVerifier [0x0x7ff73c2a6ff0+77008]
	(No symbol) [0x0x7ff73c059dea]
	(No symbol) [0x0x7ff73c0b0256]
	(No symbol) [0x0x7ff73c0b050c]
	(No symbol) [0x0x7ff73c103887]
	(No symbol) [0x0x7ff73c0d84af]
	(No symbol) [0x0x7ff73c10065c]
	(No symbol) [0x0x7ff73c0d8243]
	(No symbol) [0x0x7ff73c0a1431]
	(No symbol) [0x0x7ff73c0a21c3]
	GetHandleVerifier [0x0x7ff73c57d2cd+3051437]
	GetHandleVerifier [0x0x7ff73c577923+3028483]
	GetHandleVerifier [0x0x7ff73c5958bd+3151261]
	GetHandleVerifier [0x0x7ff73c2c185e+185662]
	GetHandleVerifier [0x0x7ff73c2c971f+218111]
	GetHandleVerifier [0x0x7ff73c2afb14+112628]
	Ge

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
 
from time import sleep
import random
import re
 
from selenium import webdriver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 1. 네이버 지도 열기
URL = "https://map.naver.com/p/search/신림동 음식점"
driver.get(URL)
time.sleep(5)

# 2. 검색 결과 iframe 진입
driver.switch_to.frame("searchIframe")
time.sleep(2)

# 3. 다음 페이지 존재 여부 확인 반복
while True:
    print("[INFO] 현재 페이지 처리 중...")

    # TODO: 여기서 음식점 정보 수집 가능

    try:
        next_btn = driver.find_element(By.CSS_SELECTOR, "a.pagination.next")
        aria_disabled = next_btn.get_attribute("aria-disabled")
        
        if aria_disabled == "true":
            print("[DONE] 마지막 페이지 도달")
            break

        next_btn.click()
        time.sleep(3)

    except Exception as e:
        print("[ERROR] 다음 페이지 버튼 찾기 실패:", e)
        break

driver.quit()

NoSuchFrameException: Message: searchIframe


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re

from selenium import webdriver
import sys

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=3)

# 반복 종료 조건
loop = True

URL = 'https://map.naver.com/p/search/%EC%9D%B8%ED%95%98%EB%8C%80%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=15.23,0,0,0,dh'
driver.get(url=URL)

while(True):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)

        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절

        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break

        last_height = new_height


    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(2)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름
        category = '' # 카테고리
        new_open = '' # 새로 오픈
        
        rating = 0.0 # 평점
        visited_review = 0 # 방문자 리뷰
        blog_review = 0 # 블로그 리뷰
        store_id = '' # 가게 고유 번호
        
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호

        switch_left()


        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()

        sleep(2)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')


        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text

        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text

        if(len(store_info) > 2):
            # 새로 오픈
            new_open = title.find_element(By.XPATH,'.//div[1]/div[1]/span[3]').text


        ###############################

        review = title.find_elements(By.XPATH,'.//div[2]/span')

        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1

        try:
            # 방문자 리뷰
            visited_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text


            # 인덱스를 다시 +1 증가 시킴
            _index += 1

            # 블로그 리뷰
            blog_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
        except:
            print(Colors.RED + '------------ 리뷰 부분 오류 ------------' + Colors.RESET)

        # 가게 id
        store_id = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]







        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text


        try:
            driver.find_element(By.XPATH,'//div[@class="y6tNq"]//span').click()

            # 영업 시간 더보기 버튼을 누르고 2초 반영시간 기다림
            sleep(2)


            parent_element = driver.find_element(By.XPATH,'//a[@class="gKP9i RMgN0"]')
            child_elements = parent_element.find_elements(By.XPATH, './*[@class="w9QyJ" or @class="w9QyJ undefined"]')

            for child in child_elements:
                # 각 자식 요소 내에서 클래스가 'A_cdD'인 span 요소 찾기
                span_elements = child.find_elements(By.XPATH, './/span[@class="A_cdD"]')

                # 찾은 span 요소들의 텍스트 출력
                for span in span_elements:
                    business_hours.append(span)
            
            # 가게 전화번호
            phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text

        except:
            print(print(Colors.RED + '------------ 영업시간 / 전화번호 부분 오류 ------------' + Colors.RESET))
        
        print(Colors.BLUE + f'{index}. ' + str(store_name) + Colors.RESET + ' · ' + str(category) + Colors.RED + str(new_open) + Colors.RESET)
        print('평점 ' + Colors.RED + str(rating) + Colors.RESET + ' / ' + visited_review + ' · ' + blog_review)
        print(f'가게 고유 번호 -> {store_id}')
        print('가게 주소 ' + Colors.GREEN + str(address) + Colors.RESET)
        print(Colors.CYAN + '가게 영업 시간' + Colors.RESET)
        for i in business_hours:
            print(i.text)
            print('')
        print('가게 번호 ' + Colors.GREEN + phone_num + Colors.RESET)
        print(Colors.MAGENTA + "-"*50 + Colors.RESET)
    
    switch_left()
        
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[3]/div[2]/a[7]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app-root"]/div/div[3]/div[2]/a[7]"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff68b906f95+76917]
	GetHandleVerifier [0x0x7ff68b906ff0+77008]
	(No symbol) [0x0x7ff68b6b9dea]
	(No symbol) [0x0x7ff68b710256]
	(No symbol) [0x0x7ff68b71050c]
	(No symbol) [0x0x7ff68b763887]
	(No symbol) [0x0x7ff68b7384af]
	(No symbol) [0x0x7ff68b76065c]
	(No symbol) [0x0x7ff68b738243]
	(No symbol) [0x0x7ff68b701431]
	(No symbol) [0x0x7ff68b7021c3]
	GetHandleVerifier [0x0x7ff68bbdd2cd+3051437]
	GetHandleVerifier [0x0x7ff68bbd7923+3028483]
	GetHandleVerifier [0x0x7ff68bbf58bd+3151261]
	GetHandleVerifier [0x0x7ff68b92185e+185662]
	GetHandleVerifier [0x0x7ff68b92971f+218111]
	GetHandleVerifier [0x0x7ff68b90fb14+112628]
	Ge